# Quickstart for Carsus

In this guide you will learn how to create atomic data files suitable for [TARDIS](https://github.com/tardis-sn/tardis).

## Atomic Weights and Ionization Energies

Download atomic weights and ionization energies from the National Institute of Standards and Technology (NIST).


<div class="alert alert-info">

**Read first:**

Get familiar with [Notation in Carsus](development/notation.rst) and learn how to select different sets of ions.
    
</div>

In [ ]:
from carsus.io.nist import NISTWeightsComp, NISTIonizationEnergies

In [ ]:
atomic_weights = NISTWeightsComp()
ionization_energies = NISTIonizationEnergies('H-Zn')

## Levels, Lines, Collisions and Cross-sections

Carsus supports three sources of energy levels and spectral lines: GFALL, CHIANTI and CMFGEN.

### GFALL

The Robert Kurucz's Atomic Linelist (GFALL) reader is the main source of **levels and lines**.

In [ ]:
!wget -qO /tmp/gfall.dat https://media.githubusercontent.com/media/tardis-sn/carsus-db/master/gfall/gfall_latest.dat

In [ ]:
from carsus.io.kurucz import GFALLReader

gfall_reader = GFALLReader('H-Zn', '/tmp/gfall.dat')

<div class="alert alert-warning">

**Warning:**
    
Creating a `GFALLReader` instance is **required**. Other sources of levels and lines are **optional**.

</div>

### CHIANTI

The Chianti Atomic Database reader provides levels and lines but also **collision strengths**.

In [ ]:
from carsus.io.chianti_ import ChiantiReader

chianti_reader = ChiantiReader('H-He', collisions=True, priority=20)

By default `priority` parameter is set to `10`. Increase this value if you want to keep CHIANTI levels and lines over GFALL.

### CMFGEN

The atomic database of [CMFGEN](http://kookaburra.phyast.pitt.edu/hillier/web/CMFGEN.htm) is a source of levels, lines and (optionally) **ionization energies**, **photoionization cross-sections** and **collisions** .

In [ ]:
from carsus.io.cmfgen import CMFGENReader

cmfgen_reader = CMFGENReader.from_config('Si 0-1', '/tmp/atomic', 
                                         priority=30,
                                         ionization_energies=True,
                                         cross_sections=True,
                                         collisions=False,
                                         temperature_grid=None,
                                         drop_mismatched_labels=True)

<div class="alert alert-warning">

**Warning:**
    
Remember that cross-sections requires data from `H 0` so give this the reader enough `priority`. 

</div>

If you want to use the **ionization energies** from CMFGEN, you will need to apply this dirty hack to merge both sources:

In [ ]:
import copy
import pandas as pd

combined_ionization_energies = copy.deepcopy(ionization_energies)
combined_ionization_energies.base = cmfgen_reader.ionization_energies.combine_first(ionization_energies.base)

## Zeta Data

Knox S. Long's ground state recombinations fractions ($\zeta$).

In [ ]:
from carsus.io.zeta import KnoxLongZeta

zeta_data = KnoxLongZeta()

## Dump to HDF5

Finally, create a `TARDISAtomData` object and dump the data with the `to_hdf` method.

In [ ]:
from carsus.io.output import TARDISAtomData

atom_data = TARDISAtomData(atomic_weights,
                           combined_ionization_energies,
                           gfall_reader,
                           zeta_data,
                           chianti_reader,
                           cmfgen_reader)

In [ ]:
atom_data.to_hdf('kurucz_cd23_chianti_He_cmfgen_H_Si_I-II.h5')

You are done! Now you can use your file to run TARDIS simulations.

## Metadata

Carsus stores metadata inside the HDF5 files to ensure reproducibility. This metadata includes a checksum per `DataFrame`, the current version of every dataset and relevant software versions. 

In [ ]:
import pandas as pd

In [ ]:
pd.read_hdf('kurucz_cd23_chianti_He_cmfgen_H_Si_I-II.h5', key='meta')

In [ ]:
# nbsphinx hidden cell
!rm kurucz_cd23_chianti_He_cmfgen_H_Si_I-II.h5